In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics
import random
from random import randint
import time
import os
import numpy as np
from collections import namedtuple
import cv2
from pathlib import Path
import argparse
import os
import csv
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense



# pip install imbalanced-learn
from imblearn.over_sampling import SMOTE

In [ ]:
### 실행결과 동일하게(완전 일치하지는 않음)
tf.keras.utils.set_random_seed(42)
### 텐서 연산 고정(완전 일치하지는 않음)
tf.config.experimental.enable_op_determinism()

In [ ]:
# data = 원본데이터 | data2 = 원본데이터 복붙 | data3 = 우리가 측정한 데이터
folder_path_1 = "./data/이상(방화)"
folder_path_2 = "./data/이상(유기)"
folder_path_3 = "./data/이상(전도)"
folder_path_5 = "./data/가방"
folder_path_6 = "./data/주머니"
folder_path_7 = "./data/이상(파손)"
folder_path_8 = "./data/이상(폭행)"
folder_path_9 = "./data/이상(흡연)"
folder_path_10 = "./data/구매(구매)"
folder_path_11 = "./data/구매(반품)"
folder_path_12 = "./data/구매(비교)"
folder_path_13 = "./data/구매(선택)"

In [ ]:
import os
import pandas as pd

def concat(path):
    file_lst = os.listdir(path)
    df_list = []

    for file in file_lst:
        file_name = os.path.join(path, file)
        df = pd.read_csv(file_name)
        df = df[df.NUMOFBODIES != 0].reset_index(drop=True)

        # 각 열을 순차적으로 뺍니다.
        for i in range(1, df.shape[1]-1):  # 첫 번째 열을 제외하고 시작
            # 변환: 문자열이 포함된 열을 숫자로 변환
            df[df.columns[i]] = pd.to_numeric(df[df.columns[i]], errors='coerce')
            df[df.columns[i+1]] = pd.to_numeric(df[df.columns[i+1]], errors='coerce')

            # 원래의 열에 연산 결과 할당
            df.loc[df.index[:-1], df.columns[i]] = df[df.columns[i]].iloc[1:].values - df[df.columns[i]].iloc[:-1].values

        # 길이가 610보다 짧은 경우 패딩 추가
        if len(df) < 610:
            pad_length = 610 - len(df)
            pad_df = pd.DataFrame(0, index=range(len(df), 610), columns=df.columns)
            df = pd.concat([df, pad_df])
        df['index_num'] = df.index

        df_list.append(df)

    return df_list


In [ ]:
file_lst = os.listdir(folder_path_3)

In [ ]:
full_data

In [ ]:
full_data = full_data.drop(full_data.columns[1:14], axis=1)
full_data.head(10)

In [ ]:
full_data

In [ ]:
data_without_label = full_data.drop(['LABEL'], axis=1)

In [ ]:
data_without_label.shape

In [ ]:
sequence_start_indices = full_data.index[full_data['index_num'] == 0].tolist()
sequence_lengths = [sequence_start_indices[i] - sequence_start_indices[i-1] for i in range(1, len(sequence_start_indices))]
sequence_lengths.insert(0, sequence_start_indices[0])

# Min sequence length
min_sequence_length = min(sequence_lengths)

In [ ]:
min_sequence_length = 610

In [ ]:
X_sequences = []

# Iterating over the start indices to create sequences
for i in range(len(sequence_start_indices) - 1):
    start_index = sequence_start_indices[i]
    end_index = sequence_start_indices[i + 1]

    # Ensure the sequence has the expected length before appending
    if end_index - start_index == min_sequence_length:
        X_sequence = data_without_label.iloc[start_index:end_index].values
        X_sequences.append(X_sequence)

# Converting lists to numpy arrays
X_sequences = np.array(X_sequences)

X_sequences.shape


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X_sequences, y_sequences, test_size=0.3, random_state=48)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=48)

In [ ]:
## 오버샘플링 (SMOTE)
#  - 2차원 이하 차원만 가능

# 학습데이터 차원 낮추기 (3차원 -> 2차원)
# X_train.shape -> (4303, 610, 32)
X_train = X_train.reshape(4303, 610 * 28)

# SMOTE를 적용할 때는 반드시 !!학습 데이터!!만 오버샘플링해야 함
smote = SMOTE(random_state=48)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

X_smote.shape, y_smote.shape

In [ ]:
# 원래대로 차원 늘리기 (2차원 -> 3차원)
# X_smote의 변경된 데이터 개수로 바꿔주기
# - 행위마다 오버샘플링 시 데이터 개수가 다름
X_smote = X_smote.reshape(7606, 610, 28)

In [ ]:
X_train.shape, X_smote.shape

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
print(X_temp.shape, y_temp.shape)

In [ ]:
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_val = X_val.astype(np.float32)
y_val = y_val.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

X_smote = X_smote.astype(np.float32)
y_smote = y_smote.astype(np.float32)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(X_smote.shape[1], X_smote.shape[2])),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 훈련 데이터에 대한 샘플 가중치 마스크 생성
# 패딩이 아닌 값은 1, 패딩된 값은 0
sample_weight_train = np.where(X_smote != 0, 1, 0)
sample_weight_train = sample_weight_train.max(axis=-1)

# 각 시퀀스에 대한 평균 샘플 가중치 계산
sample_weight_train_avg = sample_weight_train.mean(axis=1)

In [ ]:
# !!!!!행위마다 모델 이름 변경하기!!!!!
model_file_path = "./model/yugi.h5"
checkpoint_cb = keras.callbacks.ModelCheckpoint(model_file_path)
# - 자동 훈련 멈추기 함수 사용 : 추가 훈련 epoch 3회, 가중치 업데이트
# - patience : 검증 손실의 최소값에 도달한 후 중단하기 전에 기다려야 하는 epoch 수
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 5,
                                        restore_best_weights=True)


# 모델 훈련
history = model.fit(X_smote, y_smote, validation_data=(X_val, y_val), epochs=20,
                    batch_size=128, sample_weight=sample_weight_train_avg,
                    callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")


In [ ]:
# 모델 불러오기
from keras.models import load_model
model1 = load_model('./model/jeon.h5')

In [ ]:
X_sequences
X_val = X_sequences.astype(np.float32)

In [ ]:
# 테스트 데이터에 대한 예측을 한 번만 수행
test_preds = model1.predict(X_val)
test_pred_idx = [np.argmax(pred) for pred in test_preds]

In [ ]:
test_pred_idx

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# 실제 레이블과 예측 레이블을 사용하여 Confusion Matrix를 계산합니다.
conf_matrix = confusion_matrix(y_val, test_pred_idx)

# F1 Score를 계산합니다.
f1 = f1_score(y_val, test_pred_idx, average='weighted')

# Confusion Matrix를 시각화합니다.
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
# 모델에 맞게 타이틀 변경하기
plt.title('smoke smote')
plt.show()

# 결과를 출력합니다.
print("F1 Score:", f1)
print("\nClassification Report:\n", classification_report(y_val, test_pred_idx))